#### 1. Converting data

In [1]:
"""def convert_spanish_to_conll(input_path, output_path):
    with open(input_path, 'r', encoding='utf-8') as infile, \
         open(output_path, 'w', encoding='utf-8') as outfile:

        current_sentence = None

        for line in infile:
            parts = line.strip().split()
            if not parts or len(parts) < 4:
                continue 

            sentence_id = parts[0]
            # word = parts[2]
            ner_tag = parts[-1]  

            if current_sentence is None:
                current_sentence = sentence_id
            elif sentence_id != current_sentence:
                outfile.write("\n")
                current_sentence = sentence_id

            outfile.write(f"{word} {ner_tag}\n")

        outfile.write("\n")  
"""


'def convert_spanish_to_conll(input_path, output_path):\n    with open(input_path, \'r\', encoding=\'utf-8\') as infile,          open(output_path, \'w\', encoding=\'utf-8\') as outfile:\n\n        current_sentence = None\n\n        for line in infile:\n            parts = line.strip().split()\n            if not parts or len(parts) < 4:\n                continue \n\n            sentence_id = parts[0]\n            # word = parts[2]\n            ner_tag = parts[-1]  \n\n            if current_sentence is None:\n                current_sentence = sentence_id\n            elif sentence_id != current_sentence:\n                outfile.write("\n")\n                current_sentence = sentence_id\n\n            outfile.write(f"{word} {ner_tag}\n")\n\n        outfile.write("\n")  \n'

In [2]:
#convert_spanish_to_conll("ensemble.conll-2002.txt", "spanish.conll")


In [3]:
"""def convert_ewt_to_conll(input_path, output_path):
    with open(input_path, 'r', encoding='utf-8') as infile, \
         open(output_path, 'w', encoding='utf-8') as outfile:

        for line in infile:
            line = line.strip()
            if not line or line.startswith("#"):
                if line == "":
                    outfile.write("\n")  
                continue  

            parts = line.split('\t')
            if len(parts) < 3:
                continue

            token = parts[1]
            tag = parts[2]

            outfile.write(f"{token} {tag}\n")

        outfile.write("\n")"""


'def convert_ewt_to_conll(input_path, output_path):\n    with open(input_path, \'r\', encoding=\'utf-8\') as infile,          open(output_path, \'w\', encoding=\'utf-8\') as outfile:\n\n        for line in infile:\n            line = line.strip()\n            if not line or line.startswith("#"):\n                if line == "":\n                    outfile.write("\n")  \n                continue  \n\n            parts = line.split(\'\t\')\n            if len(parts) < 3:\n                continue\n\n            token = parts[1]\n            tag = parts[2]\n\n            outfile.write(f"{token} {tag}\n")\n\n        outfile.write("\n")'

In [4]:
#convert_ewt_to_conll("en_ewt-ud-train.iob2", "ewt.conll")


In [5]:
import os
import glob

def clean_and_merge_conll_files(folder_path, output_path):
    with open(output_path, 'w', encoding='utf-8') as outfile:
        files = glob.glob(os.path.join(folder_path, "*.conll"))

        for file_path in files:
            with open(file_path, 'r', encoding='utf-8') as infile:
                for line in infile:
                    line = line.strip()
                    if not line:
                        outfile.write("\n")
                        continue

                    parts = line.split()
                    if len(parts) < 2:
                        continue 

                    word = parts[0]
                    tag = parts[1]  

                    outfile.write(f"{word} {tag}\n")

                outfile.write("\n")  

    print(f"Merged {len(files)} files into {output_path}")


In [6]:
clean_and_merge_conll_files("./data/train", "combined_training.conll")

Merged 0 files into combined_training.conll


In [7]:
def load_conll_data(conll_path):
    sentences = []
    labels = []
    with open(conll_path, encoding="utf-8") as f:
        tokens = []
        tags = []
        for line in f:
            line = line.strip()
            if not line:
                if tokens:
                    sentences.append(tokens)
                    labels.append(tags)
                    tokens, tags = [], []
                continue
            token, tag = line.split()
            tokens.append(token)
            tags.append(tag)
        if tokens:  
            sentences.append(tokens)
            labels.append(tags)
    return sentences, labels


In [8]:
sentences, tags = load_conll_data(".data/combined_training.conll")

FileNotFoundError: [Errno 2] No such file or directory: '.data/combined_training.conll'

In [ ]:
def parse_conllu(file_path):
    " This function create a usable dataset of the sparse .conllu file with the NER tags and cultural refferences"
    sentences = []
    tags = []
    cultural_references = []
    
    with open(file_path, "r",encoding="utf-8") as file:
        tokens = []
        ner_tags = []
        cultural_ref = []
        
        for line in file:
            line = line.strip()
                       
            if not line:
                if tokens:  
                    sentences.append(tokens)
                    tags.append(ner_tags)
                    cultural_references.append(cultural_ref)
              
                tokens = []
                ner_tags = []
                cultural_ref = []
                continue

            parts = line.split("\t")
            
            if len(parts) >= 3:
                token = parts[0]  
                ner_tag = parts[1] 
                cultural_ref_val = parts[2]
                
                tokens.append(token)
                ner_tags.append(ner_tag)
                cultural_ref.append(cultural_ref_val)
        
       
        if tokens:
            sentences.append(tokens)
            tags.append(ner_tags)
            cultural_references.append(cultural_ref)
    
    return sentences, tags, cultural_references

In [ ]:
sentences,tags,cultural_tags= parse_conllu("./data/train_merged_output.conll")

#### 2. Labels

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit([tag for seq in tags for tag in seq])
tag2id = {tag: i for i, tag in enumerate(label_encoder.classes_)}
id2tag = {i: tag for tag, i in tag2id.items()}


In [ ]:
from transformers import XLMRobertaTokenizerFast

tokenizer = XLMRobertaTokenizerFast.from_pretrained("xlm-roberta-base")

def tokenize_and_align(sentences, tags):
    tokenized_inputs = tokenizer(
        sentences,
        is_split_into_words=True,
        return_offsets_mapping=True,
        padding=True,
        truncation=True,
        return_tensors="pt"
    )

    labels_aligned = []
    for i, label in enumerate(tags):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        aligned = []
        previous_word_idx = None
        for word_idx in word_ids:
            if word_idx is None:
                aligned.append(-100)
            elif word_idx != previous_word_idx:
                aligned.append(tag2id[label[word_idx]])
            else:
                aligned.append(-100)  # ignore subword pieces
            previous_word_idx = word_idx
        labels_aligned.append(aligned)

    tokenized_inputs["labels"] = labels_aligned
    return tokenized_inputs


c:\coding\.venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\coding\.venv\lib\site-packages\huggingface_hub\file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
from datasets import Dataset

data = [{"tokens": s, "ner_tags": t} for s, t in zip(sentences, tags)]
dataset = Dataset.from_list(data)


In [ ]:
dataset = dataset.train_test_split(test_size=0.1, seed=42)
train_dataset = dataset["train"]
val_dataset = dataset["test"]


In [ ]:
from transformers import XLMRobertaTokenizerFast

tokenizer = XLMRobertaTokenizerFast.from_pretrained("xlm-roberta-base")


c:\coding\.venv\lib\site-packages\huggingface_hub\file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
def tokenize_and_align_labels(example):
    tokenized = tokenizer(
        example["tokens"],
        is_split_into_words=True,
        truncation=True,
        padding="max_length",
        max_length=128,
    )
    word_ids = tokenized.word_ids()
    label_ids = []
    previous_word_idx = None

    for word_idx in word_ids:
        if word_idx is None:
            label_ids.append(-100)
        elif word_idx != previous_word_idx:
            label_ids.append(tag2id[example["ner_tags"][word_idx]])
        else:
            label_ids.append(-100)  
        previous_word_idx = word_idx

    tokenized["labels"] = label_ids
    return tokenized


In [ ]:
tokenized_train = train_dataset.map(tokenize_and_align_labels)
tokenized_val = val_dataset.map(tokenize_and_align_labels)

Map:   0%|          | 0/16093 [00:00<?, ? examples/s]

In [ ]:
tokenized_train[0]
tokenized_val[0]

{'tokens': ['I', 'beg', 'to', 'differ', '.'],
 'ner_tags': ['O', 'O', 'O', 'O', 'O'],
 'input_ids': [0,
  87,
  32834,
  47,
  129927,
  6,
  5,
  2,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
 

In [ ]:
for i in range(3):
    tokens = tokenizer.convert_ids_to_tokens(tokenized_train[i]['input_ids'])
    labels = [id2tag.get(id, "IGN") for id in tokenized_train[i]['labels']]
    print(list(zip(tokens, labels)))
    print("=" * 40)

[('<s>', 'IGN'), ('▁Thanks', 'O'), ('▁', 'O'), (',', 'IGN'), ('</s>', 'IGN'), ('<pad>', 'IGN'), ('<pad>', 'IGN'), ('<pad>', 'IGN'), ('<pad>', 'IGN'), ('<pad>', 'IGN'), ('<pad>', 'IGN'), ('<pad>', 'IGN'), ('<pad>', 'IGN'), ('<pad>', 'IGN'), ('<pad>', 'IGN'), ('<pad>', 'IGN'), ('<pad>', 'IGN'), ('<pad>', 'IGN'), ('<pad>', 'IGN'), ('<pad>', 'IGN'), ('<pad>', 'IGN'), ('<pad>', 'IGN'), ('<pad>', 'IGN'), ('<pad>', 'IGN'), ('<pad>', 'IGN'), ('<pad>', 'IGN'), ('<pad>', 'IGN'), ('<pad>', 'IGN'), ('<pad>', 'IGN'), ('<pad>', 'IGN'), ('<pad>', 'IGN'), ('<pad>', 'IGN'), ('<pad>', 'IGN'), ('<pad>', 'IGN'), ('<pad>', 'IGN'), ('<pad>', 'IGN'), ('<pad>', 'IGN'), ('<pad>', 'IGN'), ('<pad>', 'IGN'), ('<pad>', 'IGN'), ('<pad>', 'IGN'), ('<pad>', 'IGN'), ('<pad>', 'IGN'), ('<pad>', 'IGN'), ('<pad>', 'IGN'), ('<pad>', 'IGN'), ('<pad>', 'IGN'), ('<pad>', 'IGN'), ('<pad>', 'IGN'), ('<pad>', 'IGN'), ('<pad>', 'IGN'), ('<pad>', 'IGN'), ('<pad>', 'IGN'), ('<pad>', 'IGN'), ('<pad>', 'IGN'), ('<pad>', 'IGN'), ('<p

In [ ]:
from transformers import XLMRobertaForTokenClassification

num_labels = len(tag2id)

model = XLMRobertaForTokenClassification.from_pretrained(
    "xlm-roberta-base",
    num_labels=num_labels,
    id2label=id2tag,
    label2id=tag2id,
)

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
import numpy as np
from seqeval.metrics import precision_score, recall_score, f1_score

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [id2tag[p] for (p, l) in zip(pred, label) if l != -100]
        for pred, label in zip(predictions, labels)
    ]
    true_labels = [
        [id2tag[l] for (p, l) in zip(pred, label) if l != -100]
        for pred, label in zip(predictions, labels)
    ]

    return {
        "precision": precision_score(true_labels, true_predictions),
        "recall": recall_score(true_labels, true_predictions),
        "f1": f1_score(true_labels, true_predictions),
    }

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./xlmr-ner-model",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,0.052600,0.051670,0.837838,0.819508,0.828571
2,0.042500,0.048568,0.866419,0.851413,0.858851
3,0.025900,0.046787,0.863845,0.855971,0.859890


TrainOutput(global_step=3018, training_loss=0.06768012601040388, metrics={'train_runtime': 2611.8882, 'train_samples_per_second': 18.483, 'train_steps_per_second': 1.155, 'total_flos': 3153790136982528.0, 'train_loss': 0.06768012601040388, 'epoch': 3.0})

In [ ]:
eval_results = trainer.evaluate()

# Print the evaluation results
print(eval_results)

{'eval_loss': 0.04678715020418167, 'eval_precision': 0.8638454461821528, 'eval_recall': 0.8559708295350957, 'eval_f1': 0.85989010989011, 'eval_runtime': 14.1801, 'eval_samples_per_second': 126.163, 'eval_steps_per_second': 7.898, 'epoch': 3.0}


In [ ]:
trainer.save_model("./model")

In [ ]:
########### TESTING PHASE ###########

clean_and_merge_conll_files("./data/test/", "./data/combined_testing.conll")

Merged 6 files into ./data/combined_testing.conll


In [ ]:
test_sentences, test_tags = load_conll_data("./data/combined_testing.conll")

test_data = [{"tokens": s, "ner_tags": t} for s, t in zip(test_sentences, test_tags)]
test_dataset = Dataset.from_list(test_data)

tokenized_test = test_dataset.map(tokenize_and_align_labels)


Map:   0%|          | 0/2952 [00:00<?, ? examples/s]

In [ ]:
trainer.evaluate(eval_dataset=tokenized_test)

{'eval_loss': 0.10240750014781952,
 'eval_precision': 0.5111876075731497,
 'eval_recall': 0.42428571428571427,
 'eval_f1': 0.4637002341920375,
 'eval_runtime': 22.0239,
 'eval_samples_per_second': 134.036,
 'eval_steps_per_second': 8.4,
 'epoch': 3.0}